# Riskfolio-Lib Tutorial: 
<br><a href="https://www.kqzyfj.com/click-101359873-15150084?url=https%3A%2F%2Flink.springer.com%2Fbook%2F9783031843037" target="_blank">
        <button style="padding:10px 20px; font-size:16px; background-color: #FFA500; color:white; border:none; border-radius:5px; cursor:pointer;">
            Buy Advanced Portfolio Optimization Book on Springer
        </button>
    </a>
<br>
<br><a href='https://ko-fi.com/B0B833SXD' target='_blank'><img height='36' style='border:0px;height:36px;' src='https://cdn.ko-fi.com/cdn/kofi1.png?v=2' border='0' alt='Buy Me a Coffee at ko-fi.com' /></a> 
<br>
<br>__[Financionerioncios](https://financioneroncios.wordpress.com)__
<br>__[Orenji](https://www.linkedin.com/company/orenj-i)__
<br>__[Riskfolio-Lib](https://riskfolio-lib.readthedocs.io/en/latest/)__
<br>__[Dany Cajas](https://www.linkedin.com/in/dany-cajas/)__

## Tutorial 17: Riskfolio-Lib with MOSEK for Real Applications (612 assets and 4943 observations)

This tutorial shows how to use Riskfolio-Lib in real applications, when problems are large scale. This is a situation that banks, hedge funds, mutual funds and others institutional investors face daily.

## 1. Downloading the data:

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import warnings

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.4%}'.format

data = pd.read_csv("assets_data.csv", index_col='Dates')

Y = data.pct_change().dropna()

display(Y.head())

,A,AAPL,ABC,ABMD,ABT,ADBE,ADM,ADSK,AEE,AEP,...,WWD,X,Y,ZBRA,ASML,BMRN,CHKP,MXIM,SHPG,SIRI
Dates,,,,,,,,,,,,,,,,,,,,,
04-01-2000,-8.3842%,-8.4321%,-6.8261%,-2.3945%,-1.9714%,-8.3890%,-1.0419%,-4.8695%,0.0000%,1.1928%,...,0.4755%,-3.7328%,-0.5362%,-1.4428%,-6.6594%,6.3725%,-9.0390%,-3.6943%,-5.6250%,-6.6869%
05-01-2000,-4.7666%,1.4642%,7.7584%,1.0498%,-0.7307%,1.9773%,-1.5792%,-6.2992%,3.8685%,3.7328%,...,-2.8302%,1.0204%,-0.3703%,1.9143%,-2.7618%,-5.0691%,-1.0062%,0.2646%,5.5188%,5.0489%
06-01-2000,-5.7057%,-8.6530%,7.6008%,0.1722%,1.6572%,0.8163%,0.5349%,-6.0921%,-0.3724%,0.5682%,...,-1.2140%,0.6061%,0.5404%,-5.4143%,-6.1538%,-0.9709%,-6.8694%,2.7704%,5.6485%,2.9457%
07-01-2000,10.4035%,4.7364%,13.0097%,-0.5158%,3.9857%,4.8583%,1.5960%,11.8564%,1.4953%,1.3183%,...,0.2453%,2.6104%,2.7229%,-1.1099%,2.2697%,4.9020%,4.7156%,2.3107%,8.9109%,-1.5060%
10-01-2000,6.0575%,-1.7588%,2.6318%,14.2825%,-2.2653%,3.8610%,0.0000%,-1.1994%,-0.7366%,-0.3717%,...,-6.1271%,-0.7828%,-1.5378%,3.3669%,9.4945%,2.3364%,10.3583%,1.3802%,-0.7273%,3.9755%


## 2. Estimating Max Mean/Risk Portfolios for all Risk Measures

Instructions to install MOSEK are in this __[link](https://docs.mosek.com/9.2/install/installation.html)__, is better to install using Anaconda. Also you will need a license, I recommend you that ask for an academic license __[here](https://www.mosek.com/products/academic-licenses/)__.

In [4]:
import riskfolio as rp

# Building the portfolio object
port = rp.Portfolio(returns=Y)

# Calculating optimum portfolio

# Select method and estimate input parameters:

method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov)

# Input model parameters:

port.solvers = ['MOSEK'] # Setting MOSEK as the default solver
# if you want to set some MOSEK params use this code as an example
# import mosek
# port.sol_params = {'MOSEK': {'mosek_params': {mosek.iparam.intpnt_solve_form: mosek.solveform.dual}}}

port.alpha = 0.05 # Significance level for CVaR, EVaR y CDaR 
model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

### 2.1 Optimizing Process by Risk Measure

In [6]:
from timeit import default_timer as timer
from datetime import timedelta

# Risk Measures available:
#
# 'MV': Standard Deviation.
# 'MAD': Mean Absolute Deviation.
# 'GMD': Gini Mean Difference.
# 'MSV': Semi Standard Deviation.
# 'FLPM': First Lower Partial Moment (Omega Ratio).
# 'SLPM': Second Lower Partial Moment (Sortino Ratio).
# 'CVaR': Conditional Value at Risk.
# 'TG': Tail Gini.
# 'EVaR': Entropic Value at Risk.
# 'RLVaR': Relativistic Value at Risk.
# 'WR': Worst Realization (Minimax)
# 'CVRG': CVaR Range.
# 'TGRG': Tail Gini Range.
# 'RG': Range
# 'ADD': Average Drawdown of uncompounded cumulative returns.
# 'UCI': Ulcer Index of uncompounded cumulative returns.
# 'CDaR': Conditional Drawdown at Risk of uncompounded cumulative returns.
# 'EDaR': Entropic Drawdown at Risk of uncompounded cumulative returns.
# 'RLDaR': Relativistic Drawdown at Risk of uncompounded cumulative returns.
# 'MDD': Maximum Drawdown of uncompounded cumulative returns (Calmar Ratio).

rms = ["MV", "MAD", "GMD", "MSV", "FLPM", "SLPM",
       "CVaR", "TG", "EVaR", "RLVaR", "WR", "CVRG",
       "TGRG", "RG", "ADD", "UCI", "CDaR", "EDaR",
       "RLDaR", "MDD"]

w = {}
for rm in rms:
    start = timer()
    w[rm] = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    end = timer()
    print(rm + ' takes ',timedelta(seconds=end-start))

MV takes  0:00:00.873693
MAD takes  0:00:03.546369
GMD takes  0:00:53.611487
MSV takes  0:00:07.288727
FLPM takes  0:00:03.156797
SLPM takes  0:00:21.251270
CVaR takes  0:00:03.392545
TG takes  0:00:47.946575
EVaR takes  0:00:44.224833
RLVaR takes  0:00:39.477530
WR takes  0:00:02.745106
CVRG takes  0:00:06.037080
TGRG takes  0:01:41.218452
RG takes  0:00:05.619867
ADD takes  0:00:09.290737
UCI takes  0:00:08.175146
CDaR takes  0:00:10.455407
EDaR takes  0:00:10.071384
RLDaR takes  0:00:11.251890
MDD takes  0:00:32.627589


### 2.2 Portfolio Weights

In [8]:
w_s = pd.DataFrame([])
for rm in rms:
    w_s = pd.concat([w_s, w[rm]], axis=1)

w_s.columns = rms

display(w_s)

,MV,MAD,GMD,MSV,FLPM,SLPM,CVaR,TG,EVaR,RLVaR,WR,CVRG,TGRG,RG,ADD,UCI,CDaR,EDaR,RLDaR,MDD
A,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
AAPL,2.7531%,2.3444%,2.3937%,2.8936%,2.0164%,2.9579%,3.7962%,4.1004%,4.9723%,0.5676%,0.0000%,2.8083%,2.5458%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
ABC,2.3723%,0.7075%,2.0959%,3.6587%,1.1226%,3.8783%,2.9236%,3.6799%,10.5767%,1.8319%,0.0000%,3.1432%,1.1737%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
ABMD,0.0000%,0.0000%,0.0000%,0.0000%,0.0001%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,6.4796%,0.0000%,0.3634%,0.0000%,0.0000%,0.0000%,0.0000%
ABT,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BMRN,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
CHKP,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
MXIM,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
SHPG,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%


### 2.3 In sample CAGR by Portfolio

In [10]:
from datetime import datetime
a1 = datetime.strptime(data.index[0], '%d-%m-%Y')
a2 = datetime.strptime(data.index[-1], '%d-%m-%Y')
days = (a2-a1).days

cagr = {} 
for rm in rms:
    a = np.prod(1 + Y @ w_s[rm]) ** (360/days)-1
    cagr[rm] = [a]

cagr = pd.DataFrame(cagr).T
cagr.columns = ['CAGR']

cagr.style.format("{:.2%}").background_gradient(cmap='RdYlGn')

,CAGR
MV,28.20%
MAD,28.01%
GMD,27.79%
MSV,28.65%
FLPM,28.07%
SLPM,28.68%
CVaR,27.91%
TG,28.28%
EVaR,29.53%
RLVaR,28.27%


## 3. Estimating Min Risk Portfolios for all Risk Measures

### 3.1 Optimizing Process by Risk Measure

In [12]:
rms = ["MV", "MAD", "GMD", "MSV", "FLPM", "SLPM",
       "CVaR", "TG", "EVaR", "RLVaR", "WR", "CVRG",
       "TGRG", "RG", "ADD", "UCI", "CDaR", "EDaR",
       "RLDaR", "MDD"]

w_min = {}
obj = 'MinRisk'
for rm in rms:
    start = timer()
    w_min[rm] = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    end = timer()
    print(rm + ' takes ',timedelta(seconds=end-start))

MV takes  0:00:00.677740
MAD takes  0:00:02.851937
GMD takes  0:00:08.020652
MSV takes  0:00:04.225400
FLPM takes  0:00:02.825029
SLPM takes  0:00:11.258057
CVaR takes  0:00:02.865963
TG takes  0:00:07.510860
EVaR takes  0:00:31.074315
RLVaR takes  0:00:31.014974
WR takes  0:00:02.742492
CVRG takes  0:00:05.613470
TGRG takes  0:00:17.427885
RG takes  0:00:05.353510
ADD takes  0:00:09.743951
UCI takes  0:00:08.455249
CDaR takes  0:00:09.883562
EDaR takes  0:00:05.944837
RLDaR takes  0:00:06.690828
MDD takes  0:00:09.375070


### 3.2 Portfolio Weights

In [14]:
w_min_s = pd.DataFrame([])
for rm in rms:
    w_min_s = pd.concat([w_min_s, w_min[rm]], axis=1)

w_min_s.columns = rms

display(w_min_s)

,MV,MAD,GMD,MSV,FLPM,SLPM,CVaR,TG,EVaR,RLVaR,WR,CVRG,TGRG,RG,ADD,UCI,CDaR,EDaR,RLDaR,MDD
A,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
AAPL,0.0000%,0.1746%,0.0000%,0.0000%,0.5335%,0.0000%,0.0000%,0.0000%,2.3313%,2.7067%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
ABC,0.0000%,0.0000%,0.0000%,0.0001%,0.1824%,0.0743%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
ABMD,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
ABT,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BMRN,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
CHKP,0.0000%,0.0000%,0.0049%,0.0002%,0.0000%,0.0003%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,3.1076%,3.6153%,4.2516%,0.0000%,0.0000%,0.0000%
MXIM,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,1.9335%,4.6054%,4.4013%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
SHPG,0.0000%,0.8500%,0.3555%,0.0000%,0.4403%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%


### 3.3 In sample CAGR by Portfolio

In [16]:
from datetime import datetime
a1 = datetime.strptime(data.index[0], '%d-%m-%Y')
a2 = datetime.strptime(data.index[-1], '%d-%m-%Y')
days = (a2-a1).days

min_cagr = {} 
for rm in rms:
    a = np.prod(1 + Y @ w_min_s[rm]) ** (360/days)-1
    min_cagr[rm] = [a]

min_cagr = pd.DataFrame(min_cagr).T
min_cagr.columns = ['CAGR']

min_cagr.style.format("{:.2%}").background_gradient(cmap='RdYlGn')

,CAGR
MV,10.79%
MAD,9.87%
GMD,10.22%
MSV,10.43%
FLPM,12.26%
SLPM,11.69%
CVaR,12.30%
TG,11.65%
EVaR,10.52%
RLVaR,10.75%


### 2.3 Calculate efficient frontier

## 4. Estimating Risk Parity Portfolios for all Risk Measures

### 4.1 Optimizing Process by Risk Measure

In [19]:
rms = ["MV", "MAD", "GMD", "MSV", "FLPM", "SLPM",
       "CVaR", "TG", "EVaR", "RLVaR", "CVRG",
       "TGRG", "UCI", "CDaR", "EDaR", "RLDaR"]

b = None # Risk contribution constraints vector, when None is equally risk per asset

w_rp = {}
for rm in rms:
    start = timer()
    

    w_rp[rm] = port.rp_optimization(model=model, rm=rm, rf=rf, b=b, hist=hist)
    end = timer()
    print(rm + ' takes ',timedelta(seconds=end-start))

MV takes  0:00:00.693707
MAD takes  0:00:03.228654
GMD takes  0:00:08.651383
MSV takes  0:00:03.879751
FLPM takes  0:00:08.101861
SLPM takes  0:00:13.003332
CVaR takes  0:00:10.629533
TG takes  0:00:57.481325
EVaR takes  0:00:32.069465
RLVaR takes  0:00:29.163052
CVRG takes  0:00:21.625211
TGRG takes  0:02:09.212534
UCI takes  0:00:09.345083
CDaR takes  0:00:06.942783
EDaR takes  0:00:06.159286
RLDaR takes  0:00:08.848305


### 4.2 Portfolio Weights

In [21]:
w_rp_s = pd.DataFrame([])
for rm in rms:
    w_rp_s = pd.concat([w_rp_s, w_rp[rm]], axis=1)

w_rp_s.columns = rms

display(w_rp_s)

,MV,MAD,GMD,MSV,FLPM,SLPM,CVaR,TG,EVaR,RLVaR,CVRG,TGRG,UCI,CDaR,EDaR,RLDaR
A,0.1246%,0.1182%,0.1183%,0.1220%,0.1132%,0.1205%,0.1279%,0.1276%,0.1482%,0.1553%,0.1283%,0.1318%,0.0625%,0.0592%,0.0729%,0.0754%
AAPL,0.1586%,0.1494%,0.1511%,0.1630%,0.1587%,0.1682%,0.1826%,0.1786%,0.2485%,0.2982%,0.1675%,0.1689%,0.1111%,0.0917%,0.1109%,0.1099%
ABC,0.2385%,0.2348%,0.2343%,0.2475%,0.2587%,0.2565%,0.2440%,0.2545%,0.2922%,0.2985%,0.2432%,0.2451%,0.1808%,0.1727%,0.1797%,0.1755%
ABMD,0.1326%,0.1215%,0.1246%,0.1272%,0.1280%,0.1301%,0.1373%,0.1308%,0.1564%,0.1611%,0.1391%,0.1367%,0.1298%,0.1085%,0.0901%,0.0887%
ABT,0.2834%,0.2660%,0.2689%,0.2686%,0.2643%,0.2701%,0.2652%,0.2790%,0.3321%,0.3682%,0.2968%,0.3061%,0.2634%,0.2585%,0.3459%,0.3636%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BMRN,0.1431%,0.1349%,0.1364%,0.1411%,0.1373%,0.1431%,0.1583%,0.1623%,0.1278%,0.1111%,0.1500%,0.1556%,0.1829%,0.1238%,0.0970%,0.0855%
CHKP,0.1515%,0.1365%,0.1388%,0.1643%,0.1359%,0.1658%,0.1890%,0.2022%,0.2059%,0.2024%,0.1543%,0.1670%,0.2244%,0.2105%,0.2744%,0.2628%
MXIM,0.1452%,0.1224%,0.1260%,0.1530%,0.1185%,0.1524%,0.1804%,0.1971%,0.2852%,0.3227%,0.1532%,0.1679%,0.0852%,0.0725%,0.0817%,0.0788%
SHPG,0.2129%,0.2091%,0.2085%,0.2068%,0.2092%,0.2089%,0.2193%,0.2150%,0.2489%,0.2948%,0.2219%,0.2219%,0.1254%,0.1131%,0.1295%,0.1435%


### 4.3 In sample CAGR

In [23]:
rp_cagr = {} 
for rm in rms:
    a = np.prod(1 + Y @ w_rp_s[rm]) ** (360/days)-1
    rp_cagr[rm] = [a]

rp_cagr = pd.DataFrame(rp_cagr).T
rp_cagr.columns = ['CAGR']

rp_cagr.style.format("{:.2%}").background_gradient(cmap='RdYlGn')

,CAGR
MV,12.93%
MAD,12.73%
GMD,12.79%
MSV,12.92%
FLPM,12.95%
SLPM,13.02%
CVaR,13.13%
TG,13.14%
EVaR,13.10%
RLVaR,12.97%
